In [2]:
import cv2
import numpy as np

# The paths of all the images
image_paths = [r'C:\Users\Priyanshu Banik\Desktop\Coding Stuff\UAS Round 2\UAS Task Round 2\1.png',
               r'C:\Users\Priyanshu Banik\Desktop\Coding Stuff\UAS Round 2\UAS Task Round 2\2.png',
               r'C:\Users\Priyanshu Banik\Desktop\Coding Stuff\UAS Round 2\UAS Task Round 2\3.png'
               ]

In [3]:
# Constants
lower_red = np.array([0,255,255])
upper_red = np.array([10,255,255])

lower_blue = np.array([100,255,255])
upper_blue = np.array([140,255,255])

lower_brown = np.array([10, 100, 20])
upper_brown = np.array([20, 255, 200])

lower_green = np.array([40, 40, 40])
upper_green = np.array([70, 255, 255])

In [4]:
results = []

In [5]:
# necessary user defined functions
def image_out(image, image_name = 'Image'):

    cv2.imshow(image_name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows


In [6]:
def count_houses(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    house_count = 0

    # Loop over the contours to identify triangles
    for contour in contours:
        # Approximate the contour to reduce the number of points
        epsilon = 0.005* cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # If the approximated contour has 3 vertices, it's a triangle
        if len(approx) == 3:
            house_count += 1
    
    return house_count

In [7]:
image_counter = 1
for path in image_paths:

    image = cv2.imread(path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    mask_red = cv2.inRange(hsv_image, lower_red, upper_red)
    mask_blue = cv2.inRange(hsv_image, lower_blue, upper_blue)
    mask_brown = cv2.inRange(hsv_image, lower_brown, upper_brown) # mask for the burnt grass
    mask_green = cv2.inRange(hsv_image, lower_green, upper_green) # mask for the unburnt grass

    red_b = count_houses(mask_red + mask_green)
    red_ub = count_houses(mask_red + mask_brown)

    blue_b = count_houses(mask_blue + mask_green)
    blue_ub  = blue_b = count_houses(mask_blue + mask_brown)

    n_house = [red_b+blue_b,red_ub+blue_ub]

    priority_b = 1*red_b + 2*blue_b
    priority_ub = 1*red_ub + 2*blue_ub

    ratio = priority_b/priority_ub

    results.append([f'image {image_counter}', n_house, [priority_b, priority_ub], ratio])
    image_counter += 1

    output_image = image.copy()
    output_image[mask_brown != 0] = [13, 218, 253]  # Overlay burnt grass in yellow
    output_image[mask_green != 0] = [255, 255, 0]  # Overlay unburnt grass in cyan
    output_image_path = path.split('.')[0] + '_output.png'
    cv2.imwrite(output_image_path, output_image)


In [8]:
n_houses = [i[1] for i in results] 
priority_houses = [i[2] for i in results]
priority_ratio = [i[3] for i in results]

priority_dict = {x[0]:x[3] for x in results}
sorted_dict = {k: v for k, v in sorted(priority_dict.items(), key=lambda item: item[1], reverse=True)}
image_by_rescue_ratio = [k for k in sorted_dict]

print(f'''
Number of Houses in Burnt and Unburnt grass in each image:
{n_houses}
Priority of Houses in Burnt and Unburnt grass in each image:
{priority_houses}
Priority Ratio of Houses in Burnt and Unburnt grass in each image:
{priority_ratio}
Images by descending order of priority:
{image_by_rescue_ratio}
''')


In [14]:
file_path = r'1.png'
# file_path = image_paths[0]

image = cv2.imread(file_path)
image_out(image)
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
mask_red = cv2.inRange(hsv_image, lower_red, upper_red)
mask_blue = cv2.inRange(hsv_image, lower_blue, upper_blue)
mask_brown = cv2.inRange(hsv_image, lower_brown, upper_brown) # mask for the burnt grass
mask_green = cv2.inRange(hsv_image, lower_green, upper_green) # mask for the unburnt grass

Mask = mask_blue + mask_brown

masked_image = cv2.bitwise_and(image, image, mask=Mask)
image_out(masked_image)

contours, _ = cv2.findContours(Mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

triangle_count = 0

# Step 5: Loop over the contours to identify triangles
for contour in contours:
    # Approximate the contour to reduce the number of points
    epsilon = 0.005* cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # If the approximated contour has 3 vertices, it's a triangle
    if len(approx) == 3:
        triangle_count += 1
        cv2.drawContours(masked_image, [approx], 0, (0, 255, 0), 2)   #Optionally draw the triangle

# Step 6: Display the number of triangles found
image_out(masked_image)
print(f"Number of houses found: {triangle_count}")

output_image = image.copy()
output_image[mask_brown != 0] = [13, 218, 253]  # Overlay burnt grass in yellow
output_image[mask_green != 0] = [255, 255, 0]  # Overlay unburnt grass in cyan
image_out(output_image)


Number of houses found: 4
